In [1]:
import sys
sys.path.append('/kaggle/input/pretrainedmodels/pretrainedmodels-0.7.4')
sys.path.append('/kaggle/input/efficientnet-pytorch/EfficientNet-PyTorch-master')
sys.path.append('/kaggle/input/pytorch-image-models/pytorch-image-models')
sys.path.append('/kaggle/input/segmentation-models-pytorch/segmentation_models_pytorch')
import numpy as np
import cv2
import pandas as pd
from glob import glob
import torch.nn as nn
import json
from torch.utils.data import Dataset, DataLoader
from torchmetrics import AveragePrecision
import torch
import torchvision
from torchvision.transforms import transforms
import albumentations as A 
from albumentations.pytorch.transforms import ToTensorV2
import os
from PIL import Image
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import torch.nn.functional as F
import time
import base64
import typing as t
import zlib
import segmentation_models_pytorch as smp

class CFG:
    data_path = '/kaggle/input/hubmap-hacking-the-human-vasculature/'
    batch_size = 1
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    th = 0.15
    chepoint_dir = '/kaggle/input/models-hubmap-vasculative/'
    model_types = ['UnetPlusPlus']
    encoder_name_list = ['efficientnet-b7']
    is_tta = True
    size = 512
    org_size = 512
#     encoder_depth = 4
#     decoder_channels = [512, 256, 128, 64]
    
    test_aug = [
        A.Resize(size, size),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensorV2()
    ]

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [2]:
%%capture
!mkdir /kaggle/working/packages
!cp -r /kaggle/input/pycocotools/* /kaggle/working/packages
os.chdir("/kaggle/working/packages/pycocotools-2.0.6/")
!python setup.py install
!pip install . --no-index --find-links /kaggle/working/packages/
os.chdir("/kaggle/working")
from pycocotools import _mask as coco_mask

In [3]:
class HUBMAPDataset(Dataset):
    def __init__(self, image_dir, transform=None):
        self.image_dir = image_dir
        self.img_list = os.listdir(image_dir)
        self.transform = transform

    def __len__(self):
        return len(self.img_list)

    def __getitem__(self, idx):
        # Load image
        image_path = os.path.join(self.image_dir, self.img_list[idx])
        image = cv2.imread(image_path)  

        if self.transform:
            data = self.transform(image=image)
            image = data['image']
        return self.img_list[idx][:-4],image

In [4]:
class EnsembleModel:
    def __init__(self):
        self.models = []

    def __call__(self, x):
        outputs = [model(x) for model in self.models]
        outputs = torch.stack(outputs, dim=0)
        avg_preds = torch.mean(outputs, dim=0)
        return avg_preds

    def add_model(self, model):
        self.models.append(model)

def build_ensemble_model():
    model = EnsembleModel()
    model_types = CFG.model_types
    encoder_name_list = CFG.encoder_name_list
    for _type, encoder_name in zip(model_types, encoder_name_list):
        model_dir = CFG.chepoint_dir + _type + '/' + encoder_name + '/'
        model_list = os.listdir(model_dir)
        for i in range(len(model_list)):
            if not model_list[i].endswith('.pth'):
                continue
            _model = smp.create_model(
                    _type, encoder_name=encoder_name, in_channels=3, classes=1, encoder_weights=None,
                )
#             if _type == 'Unet':
#                 _model = smp.Unet(encoder_name=encoder_name, activation='sigmoid', encoder_depth=CFG.encoder_depth, decoder_channels=CFG.decoder_channels, encoder_weights=None)
#             elif _type == 'PSP':
#                 _model = smp.PSPNet(encoder_name=encoder_name, activation='sigmoid', encoder_depth=CFG.encoder_depth, decoder_channels=CFG.decoder_channels, encoder_weights=None)
#             elif _type == 'FPN':
#                 _model = smp.FPN(encoder_name=encoder_name, activation='sigmoid', encoder_depth=CFG.encoder_depth, decoder_channels=CFG.decoder_channels, encoder_weights=None)
#             elif _type == 'PAN':
#                 _model = smp.PAN(encoder_name=encoder_name, activation='sigmoid', encoder_depth=CFG.encoder_depth, decoder_channels=CFG.decoder_channels, encoder_weights=None)
#             elif _type == 'UnetPlusPlus':
#                 _model = smp.UnetPlusPlus(encoder_name=encoder_name, activation='sigmoid', encoder_depth=CFG.encoder_depth, decoder_channels=CFG.decoder_channels, encoder_weights=None)
            _model.to(CFG.device)
            model_path = model_dir + model_list[i]
            print(model_path)
            _model.load_state_dict(torch.load(model_path, map_location=CFG.device))
            _model.eval()
            model.add_model(_model)
    return model

In [5]:
def TTA(x: torch.Tensor, model: nn.Module):
    # x.shape=(batch,c,h,w)
    shape = x.shape
    x = [x, *[torch.rot90(x, k=i, dims=(-2, -1)) for i in range(1, 4)]]
    x = torch.cat(x, dim=0)
    x = model(x)
    x = x.reshape(4, shape[0], 1, *shape[-2:])
    x = [torch.rot90(x[i], k=-i, dims=(-2, -1)) for i in range(4)]
    x = torch.stack(x, dim=0)
    return x.mean(0)

In [6]:
class Test:
    def encode_binary_mask(self, mask: np.ndarray) -> t.Text:
      """Converts a binary mask into OID challenge encoding ascii text."""

      # check input mask --
      if mask.dtype != bool:
        raise ValueError(
            "encode_binary_mask expects a binary mask, received dtype == %s" %
            mask.dtype)

      mask = np.squeeze(mask)
      if len(mask.shape) != 2:
        raise ValueError(
            "encode_binary_mask expects a 2d mask, received shape == %s" %
            mask.shape)

      # convert input mask to expected COCO API input --
      mask_to_encode = mask.reshape(mask.shape[0], mask.shape[1], 1)
      mask_to_encode = mask_to_encode.astype(np.uint8)
      mask_to_encode = np.asfortranarray(mask_to_encode)

      # RLE encode mask --
      encoded_mask = coco_mask.encode(mask_to_encode)[0]["counts"]

      # compress and base64 encoding --
      binary_str = zlib.compress(encoded_mask, zlib.Z_BEST_COMPRESSION)
      base64_str = base64.b64encode(binary_str)
      return base64_str
    
    
    def encode_output(self,outputs,idx):
        blood_vessel = torch.argmax(outputs, 1) 
        blood_vessel = blood_vessel == 1
        blood_vessel = blood_vessel * 1
    
        blood_vessel = blood_vessel.cpu().numpy()
        all_encode = {} 
        for i in range(blood_vessel.shape[0]):
            list_encode = []
            sliceImage = blood_vessel[i,:,:]
            binarized = sliceImage > 0
            coded_len = self.encode_binary_mask(binarized)
            list_encode.append(coded_len)
            all_encode[idx[i]] =list_encode
        return all_encode

    
   
    def get_test_transforms(self):
        return A.Compose(CFG.test_aug)
    
    def test_dataloader(self,image_folder):
        dataset = HUBMAPDataset(image_dir=image_folder, 
                                transform=self.get_test_transforms())
        return DataLoader(dataset, batch_size=CFG.batch_size,shuffle=False, num_workers=4)
    
    

    def evaluate(self,model):
        ids = []
        heights = []
        widths = []
        prediction_strings = []
        sample = None
        with torch.no_grad():
            test_dataloader = self.test_dataloader(CFG.data_path + 'test/')
            bar = tqdm(enumerate(test_dataloader), total=len(test_dataloader))
            
            for step, (idx, images) in bar:
                images = images.to(CFG.device)
                if CFG.is_tta:
                    pred = TTA(images, model)
                else:
                    pred = model(images)
                pred = F.interpolate(pred, size=[CFG.org_size, CFG.org_size], mode='bilinear', align_corners=False)
                if sample is None: sample=pred
                pred_string = ''
                pred = (pred > CFG.th).float().cpu().numpy()
                
                for m in range(len(pred)):
                    kernel = np.ones(shape=(3, 3), dtype=np.uint8)
                    binary_mask = cv2.dilate((pred[m][0] * 255), kernel, 3)
                    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(binary_mask.astype(np.uint8))
                    for i in range(1, num_labels):
                        mask_i = np.zeros_like(binary_mask)
                        mask_i[labels == i] = 1
                        mask = mask_i[:, :, np.newaxis].astype(bool)
                        score = 1.0
                        encoded = self.encode_binary_mask(mask)
                        if i==0:
                            pred_string += f"0 {score} {encoded.decode('utf-8')}"

                        else:
                            pred_string += f" 0 {score} {encoded.decode('utf-8')}"
                b, c, h, w = images.shape
                ids.append(idx[0])
                heights.append(h)
                widths.append(w)
                prediction_strings.append(pred_string)
        return ids, heights, widths, prediction_strings, sample
        


In [7]:
# top10 = [sample[i].detach().permute(1,2,0).cpu().numpy() for i in range(min(10,len(sample)))]
# img = 0
# for i in top10:
#     img += i
#     img = np.clip(img, 0, 1)
# plt.imshow(img)
# plt.show()

In [8]:
model = build_ensemble_model()

/kaggle/input/models-hubmap-vasculative/UnetPlusPlus/efficientnet-b7/1.pth


In [9]:
test = Test()
ids, heights, widths, prediction_strings, sample=test.evaluate(model)

/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


  0%|          | 0/1 [00:00<?, ?it/s]

In [10]:
submission = pd.DataFrame()
submission['id'] = ids
submission['height'] = heights
submission['width'] = widths
submission['prediction_string'] = prediction_strings
submission = submission.set_index('id')
submission.to_csv("submission.csv")
!cat submission.csv

id,height,width,prediction_string
72e40acccadf,512,512, 0 1.0 eNpLTTSwSrM38jP0B0EDAyA2NADSYBachAIYB0z7G/mDsJFPWFy8OQBSkRBu 0 1.0 eNqLDco0s02yN/E28TX2NfI39INgAwNDIIaRBggmkPQHS/sZArGRrzEQm3ib+iTkphoAAMZxEl4= 0 1.0 eNrLT40yMsmxNzAw8Dc0AJP+BoZwHhwARRFiIBqmGpmHptzQwB/CAjEMIQZDeGDayA8kZuibn5lqAgDCiB2/ 0 1.0 eNpLCY8xN860t3Y38zb2NfIzNDDwh2N/AwMgBtKGIBrIMYCIAAk/Qz9jX2M/IzA28TX2MfUKjzQCANxVEpw= 0 1.0 eNqLjwgyNU2zN/c09TeEQAMDBIbxobSRH1QGBQAFDP0htIGhgT+YAxaAknAFSFoMDFENQBOC2gGhIFb7Q5wBoo38QA4x9DP0M/Yz8jX2M/YOi4g1BADZBCzn 0 1.0 eNoLycg2tEi3N/My8jf0M/Q3NDDwh5AGIApIGhhASRgHiQ3hQUUgAhBg6A/FfoYg7G/kZ+QXlBFlCgB+Xxjw 0 1.0 eNoLzs80MUuzt3W1cjf3NPQ3NDAAYRgw9IeRCBYygPGxqUNgEITRyKLIpJGfob+xb0RCsBEAXPQg7A== 0 1.0 eNoLyk80M86yN/c29jXyM/Yz9DcCYgQ0MABiQwMwjcozgDDALAjwN4RyDP2MgMaADDTyNfI38AdjPxBMTEg2AAAwxBtb
